# WTB

Automatic classification of birds for this project : http://www.cabane-oiseaux.org/

## Data cooking
First, prepare the data : we need a train dataset and a validation dataset.
We use the Keras ImageDataGenerator that can build a training set from directories containing images.
Each sub directory must contains a category of bird : Keras will associate each one to a category in the model.

In [10]:
from PIL import Image
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import os

# Directory that contains images : each subdirectory contains a category of bird
DIR = '/home/kvjw3322/Documents/Prez/ML4Noobs/Notebooks/images'

# Generator for train
train_image_generator = ImageDataGenerator()
train_iterator = train_image_generator.flow_from_directory(
    DIR, # Root directory
    target_size=(150, 150), # Images will be processed to this size
    batch_size=32, # How many images are processed at the same time ?
    class_mode='categorical') # Each subdir is a category

# Generator for validation
valid_image_generator = ImageDataGenerator()
valid_iterator = valid_image_generator.flow_from_directory(DIR,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')


Found 3032 images belonging to 6 classes.
Found 3032 images belonging to 6 classes.


## RNN : layer by layer
We build the RNN, adding layers one by one

In [11]:
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.models import Model, Sequential

model = Sequential() 

# The input_shape is 150,150 to match the images size. The last dimension is '3' 
# because each pixel is represented by 3 values (RGB). 
# For grayscale images, this parameter would have been '1' (one value to store the grey level)
# We choose to use 32 filters (or feature maps) : all the filters are appyed to the source image at the same time.
# The convolution layer "learn" the values of each filter (as synaptic coefficients are in a Dense hidden layer).
# The kernel_size is the matrix size of the filter
model.add(Conv2D(filters=32, input_shape=(150, 150, 3), kernel_size=(5,5)))

# The subsampling layer get the best value (max with maxpooling) of a set of x pixels. This set comes from a 
# sub matrix of "pool_size"
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.25))

# Transform the output matrix into a vector for the Dense layer
model.add(Flatten())

model.add(Dense(1024, activation="relu"))
model.add(Dense(6, activation="softmax"))

## Let's learn !
It's time to make our network learn : we compile it (as Keras ask us to), and run the learning from our data.

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(generator=train_iterator, 
                    validation_data=valid_iterator,
                    validation_steps=10,
                    epochs=10,
                    steps_per_epoch=10,
                    verbose=2)

Epoch 1/10
 - 34s - loss: 15.3746 - acc: 0.0438 - val_loss: 15.2115 - val_acc: 0.0563
Epoch 2/10
 - 35s - loss: 15.3626 - acc: 0.0469 - val_loss: 15.4633 - val_acc: 0.0406
Epoch 3/10
 - 34s - loss: 15.3626 - acc: 0.0469 - val_loss: 15.3626 - val_acc: 0.0469
Epoch 4/10
 - 35s - loss: 15.4129 - acc: 0.0438 - val_loss: 15.1611 - val_acc: 0.0594
Epoch 5/10
 - 34s - loss: 15.2115 - acc: 0.0563 - val_loss: 15.2115 - val_acc: 0.0563
Epoch 6/10
 - 35s - loss: 15.2115 - acc: 0.0563 - val_loss: 15.3626 - val_acc: 0.0469
Epoch 7/10
 - 34s - loss: 15.3626 - acc: 0.0469 - val_loss: 15.6648 - val_acc: 0.0281
Epoch 8/10
 - 34s - loss: 15.4633 - acc: 0.0406 - val_loss: 15.1611 - val_acc: 0.0594
Epoch 9/10
 - 34s - loss: 15.2115 - acc: 0.0563 - val_loss: 15.1611 - val_acc: 0.0594
Epoch 10/10
 - 33s - loss: 15.2140 - acc: 0.0561 - val_loss: 15.4465 - val_acc: 0.0417
